# Environement Setup

In [ ]:
# @title 1. Setup Environment & Install ActionFormer
import os
import sys
from google.colab import drive

# 1. Monta Google Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Definisci il path del TUO progetto su Drive
# Adatta se necessario (es. /content/drive/MyDrive/MistakeDetection)
PROJECT_DIR = "/content/drive/MyDrive/MistakeDetection" 

if not os.path.exists(PROJECT_DIR):
    print(f"⚠️ ATTENZIONE: Non trovo la cartella {PROJECT_DIR}")
    print("Verifica il percorso esatto nel tuo Drive!")
else:
    print(f"📂 Cartella Progetto rilevata: {PROJECT_DIR}")

# 2. Setup ActionFormer in locale (nella VM di Colab, veloce e pulito)
WORKDIR = "/content/actionformer_workspace"
os.makedirs(WORKDIR, exist_ok=True)
%cd {WORKDIR}

if not os.path.exists("multi_step_localization"):
    print("📥 Clonazione ActionFormer...")
    !git clone https://github.com/CaptainCook4D/multi_step_localization.git

%cd multi_step_localization

# 3. Installazione dipendenze
print("📦 Installazione librerie...")
!pip install -r requirements.txt > /dev/null
!pip install pyyaml > /dev/null

# 4. Compilazione Kernel CUDA (Qui su Colab funzionerà!)
print("⚙️ Compilazione kernel CUDA...")
%cd ./libs/utils
!python setup.py install > /dev/null
%cd ../..

print("\n✅ Ambiente ActionFormer pronto e compilato.")

Ambiente locale rilevato.


# Setup ActionFormer

In [ ]:
# @title 2. Estrazione Feature Omnivore
import zipfile
import shutil
from tqdm import tqdm

# --- CONFIGURAZIONE PATH INPUT ---
# Cerchiamo lo zip in posizioni comuni nel tuo progetto
POSSIBLE_PATHS = [
    os.path.join(PROJECT_DIR, "_file", "omnivore.zip"),
    os.path.join(PROJECT_DIR, "data", "omnivore.zip"),
    os.path.join(PROJECT_DIR, "omnivore.zip")
]

ZIP_PATH = None
for p in POSSIBLE_PATHS:
    if os.path.exists(p):
        ZIP_PATH = p
        break

# Cartella temporanea di estrazione
LOCAL_FEAT_DIR = "/content/temp_omnivore_features"

if ZIP_PATH is None:
    print(f"❌ ERRORE: Non trovo 'omnivore.zip' nel tuo progetto.")
    print(f"   Ho cercato in: {POSSIBLE_PATHS}")
else:
    print(f"📂 Trovato Zip: {ZIP_PATH}")
    print(f"⏳ Estrazione in corso in: {LOCAL_FEAT_DIR}...")
    
    if os.path.exists(LOCAL_FEAT_DIR):
        shutil.rmtree(LOCAL_FEAT_DIR)
    os.makedirs(LOCAL_FEAT_DIR, exist_ok=True)
    
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(LOCAL_FEAT_DIR)
        
    print(f"✅ Estrazione completata. File pronti.")

🔧 Workdir: c:\Users\enric\Desktop\MistakeDetection\temp_af_build
📥 Clonazione ActionFormer...
🔎 Ricerca file setup.py...
❌ CRITICO: setup.py non trovato! Elenco contenuto scaricato:
multi_step_localization/
    .git/
        hooks/
        info/
        logs/
            refs/
                heads/
                remotes/
                    origin/
        objects/
            info/
            pack/
        refs/
            heads/
            remotes/
                origin/
            tags/
    actionformer/
    captaincook/
    captaincook_actionformer_annotations/
        combined/
        normal/
    metrics_data/
    model_outputs/
🔄 Directory ripristinata.


FileNotFoundError: Struttura repository non valida

# Features Extraction

In [ ]:
# @title 3. Estrazione Feature Omnivore
import zipfile
import shutil
from tqdm import tqdm

# --- CONFIGURAZIONE PATH INPUT ---
ZIP_NAME = "omnivore.zip"

if IS_COLAB:
    # Su Colab cerchiamo nel path specifico indicato
    # Nota: Aggiusto il path per sicurezza (MistakeDetection corretto)
    DRIVE_BASE = "/content/drive/MyDrive/MistakeDetection"
    
    # Tentativo 1: Root della cartella drive
    ZIP_PATH = os.path.join(DRIVE_BASE, ZIP_NAME)
    
    # Tentativo 2: Sottocartella data (fallback comune)
    if not os.path.exists(ZIP_PATH):
        ZIP_PATH = os.path.join(DRIVE_BASE, "data", ZIP_NAME)
else:
    # In locale: cartella /_file del progetto
    ZIP_PATH = os.path.join(ROOT_DIR, "_file", ZIP_NAME)

# Cartella temporanea di estrazione (sempre veloce)
EXTRACT_DIR = os.path.join(WORKDIR, "temp_features")

# --- ESECUZIONE ---
if not os.path.exists(ZIP_PATH):
    print(f"❌ ERRORE CRITICO: File zip non trovato!")
    print(f"   Cercato in: {ZIP_PATH}")
    print("   Verifica che il file esista e il nome sia corretto (omnivore.zip)")
else:
    print(f"📂 File Zip trovato: {ZIP_PATH}")
    print(f"⏳ Estrazione in corso in: {EXTRACT_DIR}...")
    
    # Pulisci se esiste già per evitare conflitti
    if os.path.exists(EXTRACT_DIR):
        shutil.rmtree(EXTRACT_DIR)
    os.makedirs(EXTRACT_DIR, exist_ok=True)
    
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_DIR)
        
    print(f"✅ Estrazione completata. File pronti.")

In [ ]:
# @title 3. Inferenza ActionFormer
import yaml

# --- PATH RELATIVI AL PROGETTO ---
# Annotazioni e Checkpoint (letti direttamente dal Drive)
ANNOTATION_JSON = os.path.join(PROJECT_DIR, "data", "annotation_json", "actionformer_split.json")
MODEL_CKPT = os.path.join(PROJECT_DIR, "checkpoints", "actionformer_best.pth")

# Output temporaneo inferenza
OUTPUT_INFERENCE = os.path.join(WORKDIR, "output_results")
os.makedirs(OUTPUT_INFERENCE, exist_ok=True)

# Configurazione YAML
config_data = {
    'dataset': {
        'json_file': ANNOTATION_JSON,
        'feat_folder': LOCAL_FEAT_DIR, # Cartella locale veloce
        'file_prefix': '',
        'file_ext': '.npz',
        'input_dim': 1024,             # Omnivore base
        'feat_stride': 16,
        'num_classes': 24,
        'default_fps': 30,
    },
    'eval': {
        'batch_size': 1,
        'nms_score_thres': 0.1,
    },
    'loader': {'batch_size': 1, 'num_workers': 2},
    'model': {
        'backbone_arch': [2, 2, 5],
        'regression_range': [[0, 4], [4, 8], [8, 16], [16, 32], [32, 10000]],
        'fpn_dim': 256, 'head_dim': 512, 'use_abs_pe': False, 'max_buffer_len_factor': 6.0, 'n_mha_win_size': 19
    },
    'train': {'head_dim': 512}
}

# Salva config
os.makedirs("configs", exist_ok=True)
CONFIG_PATH = "configs/run_config.yaml"
with open(CONFIG_PATH, 'w') as f:
    yaml.dump(config_data, f)

# Esecuzione
if os.path.exists(MODEL_CKPT) and os.path.exists(ANNOTATION_JSON):
    print("🚀 Avvio Inferenza ActionFormer...")
    !python eval.py {CONFIG_PATH} {MODEL_CKPT} --save_results --output_folder {OUTPUT_INFERENCE}
else:
    print(f"❌ File mancanti su Drive:")
    print(f"   JSON: {ANNOTATION_JSON} -> {os.path.exists(ANNOTATION_JSON)}")
    print(f"   CKPT: {MODEL_CKPT} -> {os.path.exists(MODEL_CKPT)}")

# Pooling, Zipping, Upload

In [ ]:
# @title 4. Pooling e Salvataggio Finale su Drive
import numpy as np
import json
import glob
import shutil

# --- CONFIGURAZIONE ---
RESULTS_JSON = os.path.join(OUTPUT_INFERENCE, "results.json")

# Cartella temporanea locale
TEMP_EMB_DIR = os.path.join(WORKDIR, "step_embeddings_temp")
if os.path.exists(TEMP_EMB_DIR): shutil.rmtree(TEMP_EMB_DIR)
os.makedirs(TEMP_EMB_DIR, exist_ok=True)

# Cartella finale su Drive
FINAL_DATA_DIR = os.path.join(PROJECT_DIR, "data")
ZIP_OUTPUT_NAME = "step_embeddings" # step_embeddings.zip

def load_npz(path):
    try:
        with np.load(path) as data:
            if 'features' in data: return data['features']
            if 'arr_0' in data: return data['arr_0']
            return data[list(data.keys())[0]]
    except: return None

if os.path.exists(RESULTS_JSON):
    print("🚀 Inizio elaborazione embedding...")
    
    with open(RESULTS_JSON, 'r') as f:
        data = json.load(f)
        preds = data.get('results', data)

    # Mappa file locali
    file_map = {os.path.basename(f).split('.')[0]: f for f in glob.glob(os.path.join(LOCAL_FEAT_DIR, "*.npz"))}
    
    count = 0
    fps, stride = 30.0, 16.0
    feat_rate = fps / stride

    for video_id, segments in tqdm(preds.items()):
        feat_path = file_map.get(video_id)
        if not feat_path:
            candidates = [f for k, f in file_map.items() if video_id in k]
            if candidates: feat_path = candidates[0]
        
        if not feat_path: continue

        full_feats = load_npz(feat_path)
        if full_feats is None: continue

        step_embeddings = []
        step_metadata = []

        for seg in segments:
            t_start, t_end = seg['segment']
            if seg['score'] < 0.1: continue

            s_idx, e_idx = int(t_start * feat_rate), int(t_end * feat_rate)
            s_idx, e_idx = max(0, s_idx), min(len(full_feats), e_idx)
            if e_idx <= s_idx: e_idx = s_idx + 1

            if s_idx < len(full_feats):
                pooled = np.mean(full_feats[s_idx:e_idx], axis=0)
                step_embeddings.append(pooled)
                step_metadata.append(seg)

        if step_embeddings:
            np.savez_compressed(
                os.path.join(TEMP_EMB_DIR, f"{video_id}_steps.npz"),
                embeddings=np.array(step_embeddings),
                metadata=step_metadata
            )
            count += 1
            
    print(f"\n✅ Generati {count} file .npz")

    # Zipping e Copy su Drive
    print(f"📦 Creazione archivio ZIP...")
    zip_path = shutil.make_archive(
        base_name=os.path.join(WORKDIR, ZIP_OUTPUT_NAME),
        format='zip',
        root_dir=TEMP_EMB_DIR
    )
    
    print(f"☁️ Upload su Drive: {FINAL_DATA_DIR}...")
    try:
        shutil.copy2(zip_path, FINAL_DATA_DIR)
        print(f"✅ SUCCESSO! File salvato in: {os.path.join(FINAL_DATA_DIR, ZIP_OUTPUT_NAME + '.zip')}")
    except Exception as e:
        print(f"❌ Errore durante l'upload: {e}")
            
else:
    print("❌ Nessun risultato inferenza trovato.")